In [ ]:
import os
from collections import Counter

import numpy as np

from datasets import load_from_disk, interleave_datasets

In [ ]:
root_dir = r"C:\Users\onurg\.cache\huggingface\datasets"
data_dir = "updated_wiki40b"

path = os.path.join(root_dir, data_dir, "long_small_en")

dataset = load_from_disk(path)
print(len(dataset))

In [ ]:
# Language pair distribution within the dataset
pair_dict = Counter(dataset["pair"])
print(pair_dict.items())

In [ ]:
# For Counter class, order of returned list of keys/values doesn't change

pair_datasets = []
for pair in pair_dict.keys():
    pair_dataset = dataset.filter(lambda example: example["pair"] == pair, num_proc=8)
    pair_datasets.append(pair_dataset)

In [ ]:
def prob_with_temp(array: np.array, temperature: float = 0.3) -> np.array:

    prob_array = array / sum(array)
    print(f" Distribution before temperature adjustment: {prob_array}")
    final_array = prob_array **  temperature / sum(prob_array **  temperature)
    print(f" Distribution before temperature adjustment: {final_array}")
    return final_array

In [ ]:
initial_array = np.array(list(pair_dict.values()))
final_array = prob_with_temp(initial_array)

In [ ]:
final_dataset = interleave_datasets(pair_datasets, probabilities=final_array, seed=42)

In [ ]:
final_pair_dict = Counter(final_dataset["pair"])
print(final_pair_dict.items())

In [ ]:
path = os.path.join(root_dir, data_dir, "final_small_en_0.3")
final_dataset.save_to_disk(path)